In [2]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
from hilbert import decode, encode
import pandas as pd
from tqdm import tqdm
import torch
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="uM0HPEvEu6OyX3dTEuB4Fihgz",
    project_name="hilbertcnn",
    workspace="romantic-brutalist",
)
import os
os.environ["COMET_API_KEY"] = "uM0HPEvEu6OyX3dTEuB4Fihgz"
hyper_params = {"batch_size": 64, "num_epochs": 5, "learning_rate": 0.0015,"trainset":"_4layer_w_volume","loss_metric":"MSE","regularizer":"L1"}
experiment.log_parameters(hyper_params)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
class HilbertImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_parquet(os.path.join(self.img_dir, annotations_file))
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, -1])
        image = np.load(img_path)
        label = self.img_labels.iloc[idx, :-1].values.astype("float64")
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
train_ds = HilbertImageDataset(annotations_file="target_next_20.parquet",img_dir="imgs/train"+hyper_params["trainset"]+"/")
test_ds = HilbertImageDataset(annotations_file="target_next_20.parquet",img_dir="imgs/test"+hyper_params["trainset"]+"/")
trainloader = DataLoader(train_ds, batch_size=hyper_params["batch_size"], shuffle=True)
testloader = DataLoader(test_ds, batch_size=hyper_params["batch_size"], shuffle=True)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/romantic-brutalist/hilbertcnn/0c3dd2d71a7b432188d328e9f81f7f2d



cuda:0


In [4]:
a,b,c = next(iter(testloader))
a.shape,b.shape,c.shape

(torch.Size([64, 5, 32, 32]), torch.Size([64, 5, 16, 16]), torch.Size([64, 8]))

In [1]:
import os
os.path.join("asd","asdd","fc")

'asd/asdd/fc'

In [9]:
import warnings
from collections import namedtuple
from functools import partial
from typing import Any, Callable, List, Optional, Tuple

import torch
import torch.nn.functional as F
from torch import nn, Tensor



__all__ = ["Inception3", "InceptionOutputs", "_InceptionOutputs", "Inception_V3_Weights", "inception_v3"]


InceptionOutputs = namedtuple("InceptionOutputs", ["logits", "aux_logits"])
InceptionOutputs.__annotations__ = {"logits": Tensor, "aux_logits": Optional[Tensor]}

# Script annotations failed with _GoogleNetOutputs = namedtuple ...
# _InceptionOutputs set here for backwards compat
_InceptionOutputs = InceptionOutputs


class InceptionA(nn.Module):
    def __init__(
        self, in_channels: int, pool_features: int, conv_block: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super().__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.branch1x1 = BasicConv2d(in_channels, 64, kernel_size=1)

        self.branch5x5_1 = BasicConv2d(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = BasicConv2d(48, 64, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = BasicConv2d(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = BasicConv2d(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = BasicConv2d(96, 96, kernel_size=3, padding=1)

        self.branch_pool = BasicConv2d(in_channels, pool_features, kernel_size=1)

    def _forward(self, x: Tensor) -> List[Tensor]:
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)
    
class InceptionB(nn.Module):
    def __init__(self, in_channels: int, conv_block: Optional[Callable[..., nn.Module]] = None) -> None:
        super().__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.branch3x3 = conv_block(in_channels, 384, kernel_size=3, stride=2)

        self.branch3x3dbl_1 = conv_block(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = conv_block(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = conv_block(96, 96, kernel_size=3, stride=2)

    def _forward(self, x: Tensor) -> List[Tensor]:
        branch3x3 = self.branch3x3(x)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.max_pool2d(x, kernel_size=3, stride=2)

        outputs = [branch3x3, branch3x3dbl, branch_pool]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)
    
class InceptionC(nn.Module):
    def __init__(
        self, in_channels: int, channels_7x7: int, conv_block: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super().__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.branch1x1 = conv_block(in_channels, 192, kernel_size=1)

        c7 = channels_7x7
        self.branch7x7_1 = conv_block(in_channels, c7, kernel_size=1)
        self.branch7x7_2 = conv_block(c7, c7, kernel_size=(1, 7), padding=(0, 3))
        self.branch7x7_3 = conv_block(c7, 192, kernel_size=(7, 1), padding=(3, 0))

        self.branch7x7dbl_1 = conv_block(in_channels, c7, kernel_size=1)
        self.branch7x7dbl_2 = conv_block(c7, c7, kernel_size=(7, 1), padding=(3, 0))
        self.branch7x7dbl_3 = conv_block(c7, c7, kernel_size=(1, 7), padding=(0, 3))
        self.branch7x7dbl_4 = conv_block(c7, c7, kernel_size=(7, 1), padding=(3, 0))
        self.branch7x7dbl_5 = conv_block(c7, 192, kernel_size=(1, 7), padding=(0, 3))

        self.branch_pool = conv_block(in_channels, 192, kernel_size=1)

    def _forward(self, x: Tensor) -> List[Tensor]:
        branch1x1 = self.branch1x1(x)

        branch7x7 = self.branch7x7_1(x)
        branch7x7 = self.branch7x7_2(branch7x7)
        branch7x7 = self.branch7x7_3(branch7x7)

        branch7x7dbl = self.branch7x7dbl_1(x)
        branch7x7dbl = self.branch7x7dbl_2(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_3(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_4(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_5(branch7x7dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch7x7, branch7x7dbl, branch_pool]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)
class InceptionD(nn.Module):
    def __init__(self, in_channels: int, conv_block: Optional[Callable[..., nn.Module]] = None) -> None:
        super().__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.branch3x3_1 = conv_block(in_channels, 192, kernel_size=1)
        self.branch3x3_2 = conv_block(192, 320, kernel_size=3, stride=2)

        self.branch7x7x3_1 = conv_block(in_channels, 192, kernel_size=1)
        self.branch7x7x3_2 = conv_block(192, 192, kernel_size=(1, 7), padding=(0, 3))
        self.branch7x7x3_3 = conv_block(192, 192, kernel_size=(7, 1), padding=(3, 0))
        self.branch7x7x3_4 = conv_block(192, 192, kernel_size=3, stride=2)

    def _forward(self, x: Tensor) -> List[Tensor]:
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)

        branch7x7x3 = self.branch7x7x3_1(x)
        branch7x7x3 = self.branch7x7x3_2(branch7x7x3)
        branch7x7x3 = self.branch7x7x3_3(branch7x7x3)
        branch7x7x3 = self.branch7x7x3_4(branch7x7x3)

        branch_pool = F.max_pool2d(x, kernel_size=3, stride=2)
        outputs = [branch3x3, branch7x7x3, branch_pool]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)


class InceptionE(nn.Module):
    def __init__(self, in_channels: int, conv_block: Optional[Callable[..., nn.Module]] = None) -> None:
        super().__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.branch1x1 = conv_block(in_channels, 320, kernel_size=1)

        self.branch3x3_1 = conv_block(in_channels, 384, kernel_size=1)
        self.branch3x3_2a = conv_block(384, 384, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3_2b = conv_block(384, 384, kernel_size=(3, 1), padding=(1, 0))

        self.branch3x3dbl_1 = conv_block(in_channels, 448, kernel_size=1)
        self.branch3x3dbl_2 = conv_block(448, 384, kernel_size=3, padding=1)
        self.branch3x3dbl_3a = conv_block(384, 384, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3dbl_3b = conv_block(384, 384, kernel_size=(3, 1), padding=(1, 0))

        self.branch_pool = conv_block(in_channels, 192, kernel_size=1)

    def _forward(self, x: Tensor) -> List[Tensor]:
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = [
            self.branch3x3_2a(branch3x3),
            self.branch3x3_2b(branch3x3),
        ]
        branch3x3 = torch.cat(branch3x3, 1)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = [
            self.branch3x3dbl_3a(branch3x3dbl),
            self.branch3x3dbl_3b(branch3x3dbl),
        ]
        branch3x3dbl = torch.cat(branch3x3dbl, 1)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)

class BasicConv2d(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, **kwargs: Any) -> None:
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv(x)
        x = self.bn(x)
        return F.leaky_relu(x, inplace=True)
    


In [37]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.Conv2d_1a_3x3 = BasicConv2d(5, 32, kernel_size=3, stride=2)
        self.Conv2d_2a_3x3 = BasicConv2d(32, 32, kernel_size=3)
        self.Conv2d_2b_3x3 = BasicConv2d(32, 192, kernel_size=3, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.Mixed_5b = InceptionA(192, pool_features=32)
        self.Mixed_5c = InceptionA(256, pool_features=64)
        self.Mixed_5d = InceptionA(288, pool_features=64)
        self.Mixed_6a = InceptionB(288)
        self.Mixed_6b = InceptionC(768, channels_7x7=128)
        self.Mixed_6c = InceptionC(768, channels_7x7=160)
        self.Mixed_6d = InceptionC(768, channels_7x7=160)
        self.Mixed_6e = InceptionC(768, channels_7x7=192)
        self.Mixed_7a = InceptionD(768)
        self.Mixed_7b = InceptionE(1280)
        self.Mixed_7c = InceptionE(2048)
        self.fc1 = nn.Linear(768, 256)
        self.fc2 = nn.Linear(256, 32)
        self.fc3 = nn.Linear(32, 8)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        x = self.Conv2d_1a_3x3(x)
        # N x 32 x 149 x 149
        x = self.Conv2d_2a_3x3(x)
        # N x 32 x 147 x 147
        x = self.Conv2d_2b_3x3(x)
        # N x 64 x 147 x 147
        x = self.maxpool1(x)
        
        # N x 64 x 73 x 73
        # N x 192 x 35 x 35
        x = self.Mixed_5b(x)
        # N x 256 x 35 x 35
        x = self.Mixed_5c(x)
        # N x 288 x 35 x 35
        x = self.Mixed_5d(x)
        x2= torch.clone(x)
        x = self.Mixed_6a(x)
        # N x 768 x 17 x 17
        x = self.Mixed_6b(x)
        # N x 768 x 17 x 17
        x = self.Mixed_6c(x)
        # N x 768 x 17 x 17
        x = self.Mixed_6d(x)
        # N x 768 x 17 x 17
        x = self.Mixed_6e(x)
        x = self.avgpool(x)
        # N x 2048 x 1 x 1
        x = self.dropout(x)
        # N x 2048 x 1 x 1
        #x2 = self.avgpool(x2)
        # N x 2048 x 1 x 1
        #x2 = self.dropout(x2)
        x = torch.flatten(x, 1)
        #x2 = torch.flatten(x2, 1)
        #x = torch.cat([x,x2],1)
        # N x 2048
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        
        
        
        return x
net = Net().to(device)



In [38]:
import torch.optim as optim

criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=hyper_params["learning_rate"])

In [39]:
def train(model,optimizer,criterion,dataloader,epoch):
    model.train()
    running_loss = 0.0
    total_loss= 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        total_loss += loss.item()

        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.8f}')
            experiment.log_metric("batch_loss", running_loss/200)
            running_loss = 0.0
    print("total loss:",total_loss/len(trainloader))
    experiment.log_metrics({"loss": total_loss/len(trainloader)}, epoch=epoch)

In [43]:
with experiment.train():
    print("Running Model Training")
    for epoch in range(10):
        train(net, optimizer, criterion, trainloader, epoch)

Running Model Training


/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,   200] loss: 0.00001568
[1,   400] loss: 0.00001935
[1,   600] loss: 0.00001814
[1,   800] loss: 0.00001646
[1,  1000] loss: 0.00001848
total loss: 1.7782919764206746e-05


KeyboardInterrupt: 

In [48]:
net

Net(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(5, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Mixed_5b): InceptionA(
    (branch1x1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch5x5_1): BasicConv2d(
      (conv): Conv2d(192, 

In [47]:
from comet_ml.integration.pytorch import log_model
model_checkpoint = {
    "epoch": epoch,
    "model_state_dict": net.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "criterion":criterion}
log_model(experiment, model_checkpoint, model_name="4_layer_w_volume_inceptionv2")


In [44]:
dataiter = iter(testloader)
pd.set_option('display.max_rows', 500)
cols = list(pd.concat([pd.read_parquet("imgs/train"+hyper_params["trainset"]+"/target_next_20.parquet"),
pd.read_parquet("imgs/test"+hyper_params["trainset"]+"/target_next_20.parquet")]).iloc[:,:-1].columns)
fcs = [[i+"_target",i+"_pred"] for i in cols]
fullcols = [item for sublist in fcs for item in sublist]
tl = []
for images, labels in dataiter:
    images = images.float().to(device)
    labels = labels.float().to(device)
    outputs = net(images)
    evaldf = pd.DataFrame(labels.cpu(),columns=[i+"_target" for i in cols]).join(pd.DataFrame(outputs.cpu().tolist(),columns=[i+"_pred" for i in cols]))
    tl.append(evaldf[fullcols])
tf = pd.concat(tl)
outputs

/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages

/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages

/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages

/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/caliban/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[-2.0948e-03,  1.2949e-03, -1.6751e-03,  2.5323e-04, -1.4569e-03,
          2.0959e-03, -1.0867e-03,  1.9293e-03],
        [-2.7205e-03,  2.3584e-03, -2.4516e-03,  1.1084e-03, -2.1599e-03,
          2.5459e-03, -1.7366e-03,  2.3676e-03],
        [-2.8981e-03,  2.4941e-03, -1.6756e-03, -4.2668e-04,  9.2305e-05,
          2.9440e-03,  1.5331e-04,  3.9577e-03],
        [-2.7792e-03,  2.4077e-03, -2.6093e-03,  3.7748e-04, -1.6751e-03,
          2.5245e-03, -7.5998e-04,  3.0079e-03],
        [-1.4529e-03,  4.7329e-04, -1.3567e-03,  6.1536e-04, -1.5599e-03,
          1.5388e-03, -9.8765e-04,  1.6696e-03],
        [-1.5847e-03,  8.1147e-04, -1.4492e-03,  7.7370e-04, -1.8984e-03,
          1.7834e-03, -9.6627e-04,  1.5284e-03],
        [-8.1979e-04,  4.5647e-04, -2.1742e-03,  2.4961e-03, -2.9153e-03,
          1.0440e-03, -1.5904e-03,  1.2491e-03],
        [-7.0731e-04,  4.9919e-06, -1.5441e-03,  2.7066e-03, -3.3062e-03,
          5.5164e-05, -1.8395e-03,  6.3239e-04],
        [-8.9446

In [42]:
tf.describe()

,Low_5_target,Low_5_pred,High_5_target,High_5_pred,Low_10_target,Low_10_pred,High_10_target,High_10_pred,Low_15_target,Low_15_pred,High_15_target,High_15_pred,Low_20_target,Low_20_pred,High_20_target,High_20_pred
count,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000
mean,-0.001336,-0.003495,0.001387,0.002386,-0.001328,0.000308,0.001413,0.001496,-0.001297,-0.001542,0.001353,0.003134,-0.001293,0.001908,0.001344,0.000684
std,0.001608,0.003304,0.001755,0.001701,0.002745,0.001451,0.002813,0.001398,0.003449,0.003528,0.003395,0.001288,0.003944,0.001404,0.003954,0.001148
min,-0.031683,-0.019887,-0.000036,-0.002640,-0.050941,-0.009558,-0.016854,-0.007229,-0.066908,-0.020411,-0.043833,-0.003564,-0.059140,-0.006530,-0.039862,-0.005927
25%,-0.001732,-0.005420,0.000389,0.001233,-0.002212,-0.000422,0.000004,0.000690,-0.002545,-0.003317,-0.000317,0.002366,-0.002748,0.001175,-0.000544,0.000067
50%,-0.000888,-0.002895,0.000913,0.002191,-0.000915,0.000387,0.000953,0.001564,-0.000901,-0.000558,0.000942,0.003112,-0.000918,0.001975,0.000937,0.000789
75%,-0.000365,-0.001229,0.001784,0.003297,0.000078,0.001099,0.002305,0.002397,0.000387,0.000921,0.002582,0.003919,0.000635,0.002713,0.002813,0.001416
max,0.000047,0.004704,0.043472,0.013630,0.025916,0.007843,0.059302,0.006537,0.036149,0.009424,0.045371,0.009758,0.036223,0.006102,0.053114,0.004334


In [45]:
tf.describe()

,Low_5_target,Low_5_pred,High_5_target,High_5_pred,Low_10_target,Low_10_pred,High_10_target,High_10_pred,Low_15_target,Low_15_pred,High_15_target,High_15_pred,Low_20_target,Low_20_pred,High_20_target,High_20_pred
count,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000
mean,-0.001336,-0.000951,0.001387,0.000600,-0.001328,-0.002176,0.001413,0.001735,-0.001297,-0.001826,0.001353,0.000720,-0.001293,-0.000968,0.001344,0.001091
std,0.001608,0.001137,0.001755,0.001075,0.002745,0.001256,0.002813,0.001412,0.003449,0.001012,0.003395,0.001581,0.003944,0.000675,0.003954,0.001348
min,-0.031683,-0.008371,-0.000036,-0.008299,-0.050941,-0.008366,-0.016854,-0.010702,-0.066908,-0.008402,-0.043833,-0.016029,-0.059140,-0.005423,-0.039862,-0.014011
25%,-0.001732,-0.001642,0.000389,-0.000012,-0.002212,-0.002797,0.000004,0.000930,-0.002545,-0.002443,-0.000317,0.000089,-0.002748,-0.001364,-0.000544,0.000419
50%,-0.000888,-0.000991,0.000913,0.000603,-0.000915,-0.001867,0.000953,0.001648,-0.000901,-0.001818,0.000942,0.000940,-0.000918,-0.000897,0.000937,0.001212
75%,-0.000365,-0.000300,0.001784,0.001230,0.000078,-0.001300,0.002305,0.002429,0.000387,-0.001214,0.002582,0.001644,0.000635,-0.000533,0.002813,0.001866
max,0.000047,0.009434,0.043472,0.009484,0.025916,0.001518,0.059302,0.019091,0.036149,0.003981,0.045371,0.008983,0.036223,0.002996,0.053114,0.008930


In [12]:
tf.describe()

,cluster_0_target,cluster_0_pred,cluster_1_target,cluster_1_pred,cluster_2_target,cluster_2_pred,cluster_3_target,cluster_3_pred,cluster_4_target,cluster_4_pred,...,cluster_15_target,cluster_15_pred,cluster_16_target,cluster_16_pred,cluster_17_target,cluster_17_pred,cluster_18_target,cluster_18_pred,cluster_19_target,cluster_19_pred
count,15830.000000,15830.00000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,...,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000
mean,0.046999,0.04577,0.043083,0.044481,0.064624,0.055648,0.072142,0.069207,0.065698,0.061821,...,0.069109,0.070583,0.047505,0.048819,0.049337,0.050004,0.038155,0.042331,0.041883,0.044945
std,0.211644,0.00000,0.203050,0.000000,0.245869,0.000000,0.258730,0.000000,0.247761,0.000000,...,0.253648,0.000000,0.212723,0.000000,0.216577,0.000000,0.191577,0.000000,0.200327,0.000000
min,0.000000,0.04577,0.000000,0.044481,0.000000,0.055648,0.000000,0.069207,0.000000,0.061821,...,0.000000,0.070583,0.000000,0.048819,0.000000,0.050004,0.000000,0.042331,0.000000,0.044945
25%,0.000000,0.04577,0.000000,0.044481,0.000000,0.055648,0.000000,0.069207,0.000000,0.061821,...,0.000000,0.070583,0.000000,0.048819,0.000000,0.050004,0.000000,0.042331,0.000000,0.044945
50%,0.000000,0.04577,0.000000,0.044481,0.000000,0.055648,0.000000,0.069207,0.000000,0.061821,...,0.000000,0.070583,0.000000,0.048819,0.000000,0.050004,0.000000,0.042331,0.000000,0.044945
75%,0.000000,0.04577,0.000000,0.044481,0.000000,0.055648,0.000000,0.069207,0.000000,0.061821,...,0.000000,0.070583,0.000000,0.048819,0.000000,0.050004,0.000000,0.042331,0.000000,0.044945
max,1.000000,0.04577,1.000000,0.044481,1.000000,0.055648,1.000000,0.069207,1.000000,0.061821,...,1.000000,0.070583,1.000000,0.048819,1.000000,0.050004,1.000000,0.042331,1.000000,0.044945


In [5]:
import json
import websocket

socket='wss://fstream.binance.com/ws/'
a=None
def on_open(ws):
    print("opened")
    subscribe_message = {
        "method": "SUBSCRIBE",
        "params":
        [
         "btcusdt@kline_1m",
         ],
         "id": 1
         }

    ws.send(json.dumps(subscribe_message))

def on_message(ws, message):
    global lt
    #print("received a message")
    print(json.loads(message))    
    msg=json.loads(message)
    #print(msg["k"]["t"])
def on_close(ws):
    print("closed connection")        

ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_close=on_close)
ws.run_forever()

opened
{'id': 1, 'result': None}
{'e': 'kline', 'E': 1668449991549, 's': 'BTCUSDT', 'k': {'t': 1668449940000, 'T': 1668449999999, 's': 'BTCUSDT', 'i': '1m', 'f': 3072110112, 'L': 3072111246, 'o': '16565.00', 'c': '16570.70', 'h': '16571.30', 'l': '16560.40', 'v': '85.200', 'n': 1135, 'x': False, 'q': '1411466.60750', 'V': '45.768', 'Q': '758207.34980', 'B': '0'}}
{'e': 'kline', 'E': 1668449992061, 's': 'BTCUSDT', 'k': {'t': 1668449940000, 'T': 1668449999999, 's': 'BTCUSDT', 'i': '1m', 'f': 3072110112, 'L': 3072111251, 'o': '16565.00', 'c': '16571.30', 'h': '16571.30', 'l': '16560.40', 'v': '85.233', 'n': 1140, 'x': False, 'q': '1412013.44490', 'V': '45.801', 'Q': '758754.18720', 'B': '0'}}
{'e': 'kline', 'E': 1668449992460, 's': 'BTCUSDT', 'k': {'t': 1668449940000, 'T': 1668449999999, 's': 'BTCUSDT', 'i': '1m', 'f': 3072110112, 'L': 3072111255, 'o': '16565.00', 'c': '16571.30', 'h': '16571.30', 'l': '16560.40', 'v': '85.340', 'n': 1144, 'x': False, 'q': '1413786.57030', 'V': '45.871', 

False